# Check dye input files
**Author: Jun Sasaki  Coded 2024-12-26  Updated 2025-04-14**<br>
Checking input files for dye simulation. Demonstration of a static plot using matplotlib and an interactive plot using hvplot.

In [ ]:
import xarray as xr
import os
from xfvcom import FvcomDataLoader, FvcomAnalyzer, FvcomPlotConfig, FvcomPlotter
from xfvcom.helpers import get_index_by_value
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from IPython.core.magic import register_cell_magic

@register_cell_magic
def skip(line, cell):
    print("This cell is skipped.")

In [ ]:
base_path = "~/Github/TB-FVCOM/goto_dye/input/input_steady/2020"
base_path = os.path.expanduser(base_path)
ncfile = "obc_dye2020.nc"
ncfile_path = os.path.join(base_path, ncfile)
ds = FvcomDataLoader(ncfile_path=ncfile_path, time_tolerance=5).ds
ds

### Retrieve a list of open boundary nodes

In [ ]:
obc_nodes =ds.obc_nodes.values
print(f"obc_nodes={obc_nodes}")

### Retrieve the node index `node_idx` corresponding to a specified `node` value and create its DataArray.

In [ ]:
node = 3137
node_idx = get_index_by_value(obc_nodes, node)
print(f"node_idx={node_idx}")
da = ds['obc_dye'].isel(nobc=node_idx)

### Plot timeseries contourf
Further costomization can be made using `(fig, ax, cbar)`. 

In [ ]:
png_dir = "PNG"; os.makedirs(png_dir, exist_ok=True)
png_file = "dye_obc.png"
png_path = os.path.join(png_dir, png_file)
cfg = FvcomPlotConfig(figsize=(10,2))
plotter = FvcomPlotter(ds, cfg)
# xlim = (da.time[0].values, da.time[-1].values)
ylim = (1, 0)
# vmin, vmax = (da.min().item(), da.max().item())
vmin, vmax = (2.5, 6)
title = f"Dye at node={node}"
contourf_kwargs=dict(vmin=vmin, vmax=vmax, levels=31)
colorbar_kwargs=dict(label="Dye")
fig, ax, cbar = plotter.ts_contourf(da, ylim=ylim, title=title, contourf_kwargs=contourf_kwargs, colorbar_kwargs=colorbar_kwargs)
fig.savefig(png_path, dpi=600, bbox_inches='tight')

## Spatial 2D plot along open boundary nodes at a specified time

In [ ]:
selected_time = '2020-01-01'
selected_time = np.datetime64(selected_time)
dye_profile = ds['obc_dye'].sel(time=selected_time, method='nearest')
siglay = ds['siglay'][:, node_idx]
obc_h = ds['obc_h']
depth = siglay * obc_h  # Determine the depth at each node
depth = depth.transpose('siglay', 'nobc') 
nobc_indices = np.tile(obc_h.nobc.values, (len(siglay), 1)) 
nobc_numbers = np.tile(obc_nodes, (len(siglay),1))
siglay_grid = np.tile(siglay.values[:, None], (1, len(obc_nodes)))

In [ ]:
plt.close()
yaxis_siglay=False
plt.figure(figsize=(10, 3))
if yaxis_siglay:
    plt.contourf(
        nobc_indices, siglay_grid, dye_profile.values, levels=20, cmap='viridis'
    )
else:
    plt.contourf(
        nobc_indices, depth, dye_profile.values, levels=20, cmap='viridis'
    )
plt.colorbar(label="Dye Concentration")
plt.xlabel("Boundary Node Index")
plt.ylabel("Depth (m)")
plt.title(f"Dye Concentration Cross-Section on {selected_time}")
plt.gca().invert_yaxis()  # 水深を正の方向に（深くなるほど下へ）
#plt.grid(True)
plt.tight_layout()
plt.show()

## Interactive timeseries plot at a specified node
Useful for checking values interactively.

In [ ]:
import holoviews as hv
import panel as pn
import hvplot.xarray
from datetime import datetime
hv.extension('bokeh')
pn.extension()

In [ ]:
def plot_time_siglay(node):
    """
    The dye concentration at the specified node is plotted as a heatmap based on time series and sigma layers (siglay).

    Parameters:
    -----------
    node(int): Specified node number
    """
    # Retrieve data at the specified node while deleting nob.
    dye_profile = ds['obc_dye'].isel(nobc=node, drop=True)  # (time, siglay)
    
    # plot with 'time' vs 'siglay'
    heatmap = dye_profile.hvplot.quadmesh(
        x='time',
        y='siglay',
        cmap='viridis',
        colorbar=True,
        title=f"Dye Concentration at Node {node} Over Time",
        xlabel='Time',
        ylabel='Sigma Layer'
    )
    
    return heatmap

In [ ]:
node_select = pn.widgets.Select(
    name='Node Index',
    options=list(range(ds.sizes['nobc'])),  # 0〜12
    value=0
)

interactive_plot1 = pn.bind(plot_time_siglay, node=node_select)

# Create a layout
layout1 = pn.Column(
    "### Dye Concentration Over Time",
    node_select,
    interactive_plot1
)
final_layout = pn.Row(layout1)
final_layout

## Interactive spatial 2D plot along open boundary nodes at the specified time

In [ ]:
def plot_node_depth(time):
    """
    Plot the dye concentration as a heatmap based on node index and water depth at the specified time.

    Parameters:
    -----------
    time(str): The specified datetime
    """
    # Convert to  np.datetime64
    time_np = np.datetime64(time)
    
    # Retrieve the data at the specified time
    dye_profile = ds['obc_dye'].sel(time=time_np)  # (siglay, nobc)
    
    # Define node_indices as a list.
    node_indices = np.arange(ds.sizes['nobc'])
    dye_profile = dye_profile.assign_coords(node_index=('nobc', node_indices))
    
    # Retrieve siglay and obc_h
    # Suppose siglay has the dimensions of (siglay, nobc) at any node
    siglay_1d = ds['siglay'].isel(nobc=0).values  # (30,)
    obc_h_values = ds['obc_h'].values  # (13,)
    
    # Determine the depth using depth = siglay * obc_h
    depth = siglay_1d[:, np.newaxis] * obc_h_values  # shape (30,13)
    
    # Add depth as a coordinate
    dye_profile = dye_profile.assign_coords(depth=(['siglay', 'nobc'], depth))

    max_depth = depth.max()
    min_depth = depth.min()
    
    # Interactive plot
    heatmap = dye_profile.hvplot.quadmesh(
        x='node_index',
        y='depth',
        cmap='viridis',
        colorbar=True,
        title=f"Dye Concentration at Time {time} by Depth",
        xlabel='Node Index',
        ylabel='Depth (m)',
        ylim=(max_depth, min_depth)
    )

    return heatmap

In [ ]:
# A widget for selecting time
time_options = [pd.to_datetime(t).strftime('%Y-%m-%d %H:%M') for t in ds['time'].values]

time_select = pn.widgets.DiscreteSlider(
    name='Time',
    options = time_options,  # list of str
    value = time_options[0]
)

# Create an interactive plot
interactive_plot2 = pn.bind(plot_node_depth, time=time_select)

# Create a layout
layout2 = pn.Column(
    "# Dye Concentration by Depth",
    time_select,
    interactive_plot2
)
final_layout = pn.Row(layout2)
final_layout

# Slice netcdf with time
To check netcdf files on PC, it is necessary to make their sizes much smaller; so slicing with time

In [ ]:
base_path = "~/Github/TB-FVCOM/goto_dye/input/input_steady/2020"
base_path = os.path.expanduser(base_path)
ncfile = "TokyoBay18_2020_wnd.nc"
ncfile_path = f"{base_path}/{ncfile}"
fvcom_wnd = FvcomDataLoader(ncfile_path=ncfile_path, time_tolerance=5)

In [ ]:
start = "2020-01-01 00:00:00"
end = "2020-01-07 00:00:00"
output_path = f"sliced_{ncfile}"
fvcom_wnd.slice_by_time(start, end).to_netcdf(output_path)

In [ ]:
fvcom_wnd.ds

In [ ]:
start = "2020-01-01 00:00:00"
end = "2020-01-07 00:00:00"
base_path = "~/Github/TB-FVCOM/goto_dye/input/input_steady/2020"
base_path = os.path.expanduser(base_path)
ncfiles = ["TokyoBay2020final_tsobc.nc", "TokyoBay2020julian_obc.nc",
           "TokyoBay2020kisarazufinal_sewer.nc", "TokyoBay2020final_river.nc",
           "TokyoBay2020final_sewer.nc"]
for ncfile in ncfiles:
    output_path = f"sliced_{ncfile}"
    ncfile_path = f"{base_path}/{ncfile}"
    fvcom = FvcomDataLoader(ncfile_path=ncfile_path, time_tolerance=5)
    fvcom.slice_by_time(start, end).to_netcdf(output_path)